In [2]:
data_file = '/FileStore/tables/i0n8w7np1490541835956/kddcup_data_10_percent-d8e1d.gz'
raw_data = sc.textFile(data_file).cache()
raw_data.first()

In [3]:

from pyspark.sql import Row

csv_data = raw_data.map(lambda l: l.split(","))
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [4]:
interactions_df = sqlContext.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

In [5]:
df = spark.read.csv('/FileStore/tables/i0n8w7np1490541835956/kddcup_data_10_percent-d8e1d.gz',inferSchema=True, )

In [6]:
df.show()

In [7]:
df.printSchema()

In [8]:
# Select tcp network interactions with more than 1 second duration and no transfer from destination
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
tcp_interactions.show()

In [9]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()



In [10]:
df = spark.createDataFrame([(1, 4), (2, 5), (3, 6)], ["A", "B"])

In [11]:
df.show()

In [12]:
from pyspark.sql import functions as F
df.withColumn('C', F.lit(0)).show()

In [13]:
df.withColumn('C', df.A * 2)

In [14]:
 df.withColumn('C', df.B > 0).show()

In [15]:
df.groupBy("A").agg(F.avg("B"), F.min("B"), F.max("B")).show()